# Visualizing pre-processing results

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import holoviews as hv

from hv_anndata import ACCESSOR as A
from hv_anndata import register
from hv_anndata import scanpy as hv_sc

register()

hv.extension("bokeh")

In [ ]:
import scanpy as sc

In [ ]:
adata = sc.datasets.pbmc68k_reduced()
adata.layers["counts"] = adata.raw.X
del adata.raw
adata

{func}`scanpy.pl.highest_expr_genes`

In [ ]:
hv_sc.highest_expr_genes(adata, layer="counts")

~~{func}`scanpy.pl.filter_genes_dispersion`~~ deprecated in favor of:

{func}`scanpy.pl.highly_variable_genes`

In [ ]:
sc.pp.highly_variable_genes(adata)
# sc.pl.highly_variable_genes(adata)

In [ ]:
d1, d2 = (
    ("variances", "variances_norm")
    if adata.uns["hvg"]["flavor"] == "seurat_v3"
    else ("dispersions", "dispersions_norm")
)

hv.Layout([
    hv.Scatter(adata, [A.var["means"]], [A.var[d], A.var["highly_variable"]]).opts(
        color=A.var["highly_variable"],
        cmap={True: "black", False: "gray"},
    )
    for d in [d2, d1]
])

{func}`scanpy.pl.scrublet_score_distribution`

TODO:
- batches

missing:
- where are the y ticks on the y axis?

In [ ]:
adata_sim = sc.pp.scrublet_simulate_doublets(adata)
sc.pp.scrublet(adata, adata_sim)
# sc.pl.scrublet_score_distribution(adata)

In [ ]:
labels = dict(
    xlabel="Doublet score",
    ylabel="Probability density",
)

hv.Layout([
    hv.Dataset(adata, [], [A.obs["doublet_score"]])
    .hist(A.obs["doublet_score"], adjoin=False)
    .opts(xlim=(0, 1), logy=True, ylim=(1, None), **labels),
    hv.Table(adata.uns["scrublet"]["doublet_scores_sim"], "scores")
    .hist("scores", adjoin=False)
    .opts(xlim=(0, 1), shared_axes=False, **labels),
]) * hv.VLine(adata.uns["scrublet"]["threshold"])